# Radial Basis Function

Σκοπός της παρούσας άσκησης είναι η πρόβλεψη ως προς το αν ένας/μία πελάτης/ισσα μίας τράπεζας θα κάνει τη χρήση της προθεσμιακής κατάθεσης ή όχι. Αυτή η πρόβλεψη θα εξαχθεί με βάση συγκεκριμένα δημογραφικά χαρακτηριστικά του/της
όπως η ηλικία, η εργασία, η εκπαίδευση, η οικογενειακή κατάσταση κ.λπ.
Για να το επιτύχουμε αυτό θα κάνουμε χρήση μίας εφαρμογής βασισμένη σε RBF. Στο κρυφό επίπεδο των νευρώνων υπολογίζονται τα κέντρα των clusters με τη χρήση του αλγορίθμου k-means.


 

**Βήμα 1ο**: Αρχικά φορτώνουμε τις απαράιτητες βιβλιοθήκες

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import math
from sklearn.metrics import accuracy_score

**Βήμα 2ο**: διάβαζουμε το αρχείο bank-full.csv και απομονώνουμε τις στήλες με τα χαρακτηριστικά που θα χρησιμοποιήσουμε (εργασία, αν έχει δανειακές υποχρεώσεις κλπ). Το αρχείο είναι διαθέσιμο [εδώ](https://raw.githubusercontent.com/nkostopoulos/StochasticsLabPublic/master/lab9/bank-full.csv) και θα πρέπει να το φορτώσετε στο colab.

In [3]:
dataset = pd.read_csv('bank-full.csv',sep=None)

C:\Users\User\AppData\Local\Temp\ipykernel_6172\3887905473.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  dataset = pd.read_csv('bank-full.csv',sep=None)


In [4]:
dataset.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [5]:
col_to_use = ['age','balance','day','duration','campaign','pdays','previous']
data = dataset.drop(col_to_use,axis=1)

In [6]:
data.head()

,job,marital,education,default,housing,loan,contact,month,poutcome,y
0,management,married,tertiary,no,yes,no,unknown,may,unknown,no
1,technician,single,secondary,no,yes,no,unknown,may,unknown,no
2,entrepreneur,married,secondary,no,yes,yes,unknown,may,unknown,no
3,blue-collar,married,unknown,no,yes,no,unknown,may,unknown,no
4,unknown,single,unknown,no,no,no,unknown,may,unknown,no


In [7]:
data = data.apply(LabelEncoder().fit_transform)

In [8]:
data.head()

,job,marital,education,default,housing,loan,contact,month,poutcome,y
0,4,1,2,0,1,0,2,8,3,0
1,9,2,1,0,1,0,2,8,3,0
2,2,1,1,0,1,1,2,8,3,0
3,1,1,3,0,1,0,2,8,3,0
4,11,2,3,0,0,0,2,8,3,0


In [9]:
print(data['job'].unique())

[ 4  9  2  1 11  5  0  7  6 10  3  8]


In [10]:
print(dataset['job'].unique())

['management' 'technician' 'entrepreneur' 'blue-collar' 'unknown'
 'retired' 'admin.' 'services' 'self-employed' 'unemployed' 'housemaid'
 'student']


In [11]:
data_rest = dataset[col_to_use]
data_rest.head()

,age,balance,day,duration,campaign,pdays,previous
0,58,2143,5,261,1,-1,0
1,44,29,5,151,1,-1,0
2,33,2,5,76,1,-1,0
3,47,1506,5,92,1,-1,0
4,33,1,5,198,1,-1,0


In [12]:
dataset2 = pd.concat([data_rest,data],axis=1)

In [13]:
dataset2.head()

,age,balance,day,duration,campaign,pdays,previous,job,marital,education,default,housing,loan,contact,month,poutcome,y
0,58,2143,5,261,1,-1,0,4,1,2,0,1,0,2,8,3,0
1,44,29,5,151,1,-1,0,9,2,1,0,1,0,2,8,3,0
2,33,2,5,76,1,-1,0,2,1,1,0,1,1,2,8,3,0
3,47,1506,5,92,1,-1,0,1,1,3,0,1,0,2,8,3,0
4,33,1,5,198,1,-1,0,11,2,3,0,0,0,2,8,3,0


**Βήμα 3ο**: Σπάμε στο dataset σε train και test dataset και κάνουμε scale με τη χρήση της συνάρτησης StandardScaler()

In [14]:
X= dataset2.drop('y',axis=1)
y= dataset2['y']

X_train, X_test, y_train, y_test= train_test_split(X,y, test_size= 0.33, random_state= 4)

In [15]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [16]:
print(X_train)

[[ 3.39736083 -0.45852779  0.14087303 ... -0.71332324 -1.50513854
   0.44672656]
 [-0.27605947  2.43941798  0.50067953 ...  0.40010942 -1.83733874
  -2.58685457]
 [ 0.66584317 -0.13326522 -0.93854645 ... -0.71332324  1.48466323
  -0.56446715]
 ...
 [-0.74701079 -0.22619739  0.62061503 ... -0.71332324  1.15246303
   0.44672656]
 [ 1.60774581 -0.42275894 -1.53822394 ...  1.51354209  0.15586244
   0.44672656]
 [ 0.28908211 -0.13627382  1.58009901 ... -0.71332324 -0.17633776
   0.44672656]]


**Βήμα 4ο**: Ορίζουμε τα κέντρα των νευρώνων με τη χρήση του k-means. Αρχικά τρέχει ο αλγόριθμος k-means για να βρούμε τα κέντρα και την τυπική απόκλιση των clusters.

In [17]:
K_cent= 8
km= KMeans(n_clusters= K_cent, max_iter= 100)
km.fit(X_train)
cent= km.cluster_centers_

In [18]:
max=0 
for i in range(K_cent):
	for j in range(K_cent):
		d= np.linalg.norm(cent[i]-cent[j])
		if(d> max):
			max= d
d= max

sigma= d/math.sqrt(2*K_cent)

Βήμα 5ο: Δημιουργούμε τον πίνακα σχέσεων F (γραμμικός συνδυασμός των Κ συναρτήσεων βάσης φj(x).

In [19]:
shape= X_train.shape
row= shape[0]
column= K_cent
F= np.empty((row,column), dtype= float)

In [20]:
for i in range(row):
  for j in range(column):
    dist= np.linalg.norm(X_train[i]-cent[j])
    F[i][j]= math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))

**Βήμα 6ο**: Βρίσκουμε τον πίνακα βάρους W για την εκπαίδευση του δικτύου

In [21]:
FTG= np.dot(F.T,F)
FTG_inv= np.linalg.inv(FTG)
fac= np.dot(FTG_inv,F.T)
W= np.dot(fac,y_train)

**Βήμα 7ο**: Ρύθμιση του πίνακα F για το test dataset

In [22]:
row= X_test.shape[0]
column= K_cent
F_test= np.empty((row,column), dtype= float)
for i in range(row):
	for j in range(column):
		dist= np.linalg.norm(X_test[i]-cent[j])
		F_test[i][j]= math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))

**Βήμα 8ο**: Ακρίβεια πρόβλεψης στο test dataset

In [23]:
prediction= np.dot(F_test,W)
prediction= 0.5*(np.sign(prediction-0.5)+1)

score= accuracy_score(prediction,y_test)
print(score.mean())

0.8876675603217158


## Ερωτήσεις

**Ερώτηση 1**: Να αναφέρετε επιγραμματικά με ποιον τρόπο λειτουργεί ένα νευρωνικό δίκτυο RBF

**Ερώτηση 2**:Με ποιούς τρόπους μπορούμε να υπολογίσουμε τα βάρη στα hidden layers; Ποιες διαφορές υπάρχουν στους τρόπους αυτούς; Ποιος παρέχει τη καλύτερη δυνατή λύση;

**Ερώτηση 3**: Επιχειρήστε να αλλάξετε την τιμή του k σε 10 και στη συνέχεια σε 50. Τι παρατηρείτε ως προς την ακρίβεια των αποτελεσμάτων;

**Ερώτηση 1**
Να αναφέρετε επιγραμματικά με ποιον τρόπο λειτουργεί ένα νευρωνικό δίκτυο RBF

Τα νευρωνικά δίκτυα συναρτήσεων πυρήνα RBF (radial basis function networks) είναι δίκτυα πρόσθιας τροφοδότησης (feedforward) που χρησιμοποιούνται σε προβλήματα επιβλεπόμενης μηχανικής μάθησης. Είναι δίκτυα εναλλακτικά των MLP, τα οποία ωστόσο υλοποιούν απεικονίσεις από το χώρο εισόδων στο χώρο εξόδων όπως και το MLP έχοντας, όμως, εντελώς διαφορετική λειτουργία κρυμμένου επιπέδου. Στο ακόλουθο σχήμα φαίνεται η δομή ενός τέτοιου νευρωνικού δικτύου.

<img src="lab9_part1_rbf_network.jpg"></img>

Τα δίκτυα αυτά περιέχουν μονάχα ένα κρυφό επίπεδο (hidden layer), οι νευρώνες του οποίου υπολογίζουν μια συνάρτηση $h_j(\vec{x})$ του διανύσματος εισόδου του δικτύου $\vec{x}$.

Αν ένα δίκτυο RBF έχει τα εξής χαρακτηριστικά:
- $d$ διάσταση εισόδου
- $Μ$ νευρώνες στο hidden layer
- $p$ διάσταση εξόδου

τότε, για είσοδο $\vec{x} = (x_1,x_2,...,x_d)^T$, και έξοδο $\vec{o}$ με συνιστώσες $o_i$, $i=1,2,...,p$, τότε αυτές υπολογίζονται ως:

$o_{i}(\vec{x})=v_{i 0}+\sum_{j=1}^{M} v_{i j} h_{j}(\vec{x})$

με $v_{ij}$ να είναι το βάρος που συνδέει τον νευρώνα $j$, ($j=1,2,...,M$) του hidden layer με το νευρώνα εξόδου $i$, ($i=1,2,...,p$) και $v_{i0}$ είναι η πόλωση/bias του νευρώνα εξόδου $i$. Οι νευρώνες εξόδου είναι τυπικοί νευρώνες εσωτερικού γινομένου κατ' αναλογία με το MLP με γραμμική συνάρτηση ενεργοποίησης.

Οι συναρτήσεις RBF υπεισέρχονται στους κρυμμένους νευρώνες, οι οποίοι υπολογίζουν τις συναρτήσεις $h(\vec{x})$, οι οποίες λέγονται **ακτινικές συναρτήσεις βάσης (radial basis functions)**.
Οι συναρτήσεις αυτές έχουν την εξής μορφή: υπάρχει ένα σημείο το οποίο λέγεται κέντρο και όπου οι συναρτήσεις αυτές παρέχουν μέγιστη τιμή, ενώ απομακρυνόμενοι από το σημείο αυτό η τιμή της συνάρτησης μειώνεται αρκετά γρήγορα.

Παράδειγμα ακτινικής συνάρτησης βάσης είναι η συνάρτηση Gauss, γνωστή και ως πυρήνας (kernel) RBF.
$h_{j}(\vec{x})=\exp \left(-\frac{\left\|\vec{x}-\vec{w_{j}}\right\|^{2}}{2 \sigma_{j}^{2}}\right)=\exp \left(-\frac{\sum_{i=1}^{d}\left(x_{i}-w_{ji}\right)^{2}}{2 \sigma_{j}^{2}}\right)$,

όπου το δίανυσμα βαρών $\vec{w_j}=(w_{j1}, w_{j2}, ..., w_{jd})^T$ καθορίζει το κέντρο της συνάρτησης $h_{j}(\vec{x})$, ενώ η παράμετρος $\sigma_{j}$ καθορίζει την ακτίνα της συνάρτησης. Η συνάρτηση μεγιστοποιείται στο κέντρο για $\vec{x}=\vec{w_j}$ με μέγιστη τιμή $h_j(\vec{x}=\vec{w_j})=exp(0)=1$, με την τιμή της να ελατώνεται εκθετικά απομακρυνόμενοι από το κέντρο. Η ελάττωση αυτή καθορίζεται από την τιμή της ακτίνας $\sigma_{j}$, όπου για μικρές τιμές της ο ρυθμός μείωσης είναι μεγάλος ενώ το αντίθετο συμβαίνει για μεγάλες τιμές της ακτίνας.

Συνεπώς, κάθε συνάρτηση RBF $h_{j}(\vec{x})$ χαρακτηρίζεται από μια σφαιρική περιοχή (υπερσφαίρα στις $d$ διαστάσεις) εμβέλειας, εκτός της οποίας η συνάρτηση παίρνει αμελητέες τιμές. Όταν, λοιπόν, ένα διάνυσμα εισόδου $\vec{x}$ μπει ως είσοδος στο δίκτυο RBF, οι hidden neurons που περιέχουν το $\vec{x}$ στην περιοχή εμβέλειάς τους, θα δώσουν μεγάλη τιμή εξόδου.

Αν σε ένα πρόβλημα, είναι γνωστό πως τα δεδομένα εκπαίδευσης για πρόβλημα ταξινόμησης σχηματίζουν κατά προσέγγιση σφαιρικές και διακριτές ομάδες, και τα δεδομένα κάθε ομάδας είναι της ίδιας κατηγορίας, τότε για κάθε ομάδα δεδομένων, μπορεί να οριστεί ένας νευρώνας RBF με κέντρο το κέντρο της αντίστοιχης ομάδας και ακτίνα τη διασπορά της ομάδας. Τότε για μια είσοδο $\vec{x}$, η τιμή της εξόδου $h_j(\vec{x})$ του κρυφού νευρώνα $j$ δηλώνει αν το $\vec{x}$ ανήκει στην ομάδα $j$. Άρα το δεδομένο εισόδου $\vec{x}$ απεικονίζεται στο διάνυσμα $\vec{h}(\vec{x})=(h_1(\vec{x}), h_2(\vec{x}), ..., h_M(\vec{x}))$, που δείχνει το βαθμό στον οποίο το x ανήκει σε κάποια απ' τις ομάδες. Εν συνεχεία, είναι εύκολο να καθοριστούν τα βάρη του επιπέδου εξόδου, ώστε ο κρυμμένος νευρώνας που αντιστοιχεί σε στην ομάδα κατηγορίας έστω $C_k$ να διεγείρει μόνο τον αντίστοιχο νευρώνα εξόδου $k$.

Συμπερασματικά, οι **παράμετροι/βάρη** του δικτύου RBF ου πρέπει να καθοριστούν **κατά τη διάρκεια εκπαίδευσης** του δικτύου είναι:
- **$M \cdot d$**: τα κέντρα δηλαδή $\vec{w}_j=(w_{j1}, w_{j2}, ..., w_{jd})T$ των ακτινικών συναρτήσεων βάσης, άρα των αντίστοιχων νευρώνων RBF του κρυφού επιπέδου
- **$M$**: οι ακτίνες δηλαδή $\sigma_j$  των ακτινικών συναρτήσεων βάσης, άρα των αντίστοιχων νευρώνων RBF του κρυφού επιπέδου
- **$(Μ+1)\cdot p$**: Οι παράμετροι δηλαδή του επιπέδου εξόδου συμπεριλαμβανομένων των biases των νευρώνων εξόδου

**ΣΥΝΟΛΟ ΠΑΡΑΜΕΤΡΩΝ ΠΡΟΣ ΕΚΤΙΜΗΣΗ**: $M \cdot d + M + (M+1) \cdot p = M \cdot (d+p+1) + p$


**Ερώτηση 2**
Με ποιούς τρόπους μπορούμε να υπολογίσουμε τα βάρη στα hidden layers; Ποιες διαφορές υπάρχουν στους τρόπους αυτούς; Ποιος παρέχει τη καλύτερη δυνατή λύση;

Τα δίκτυα RBF έχουν δύο τύπους βαρών/παραμέτρων προς εκμάθηση.
- **Τα βάρη $\vec{w}$** που αναπαριστούν τις συντεταγμένες των κέντρων των κρυμμένων νευρώνων
- **Τα βάρη $\vec{v}$** που είναι τα κλασικά βάρη των νευρώνων στο επίπεδο εξόδου όπως και σε ένα MLP και τα οποία ορίζουν την εξίσωση υπερεπιπέδου για τους νευρώνες εξόδου.

Για την εκτίμηση των παραμέτρων που αναφέρθηκαν παραπάνω, υπάρχουν δύο μέθοδοι εκπαίδευσης:
- **Ενιαία εκπαίδευση** για όλους τους τύπους παραμέτρων/βαρών
- **Εκπαίδευση δύο σταδίων**

Στην **ενιαία εκπαίδευση**, ορίζοντας τη συνάρτηση σφάλματος (πχ µέσο τετραγωνικό σφάλµα), μπορούμε έπειτα να εφαρμόσουμε Gradient descent. Τα βάρη $\vec{v}$ αρχικοποιούνται τυχαία όπως σε ένα MLP. Ωστόσο αν τα βάρη $\vec{w}$ των κέντρων των κρυφών νευρώνων (δηλαδή των συναρτήσεων ακτινικής βάσης) αρχικοποιούνται τυχαία, τότε η μέθοδος gradient descent είναι δύσκολο να αποφύγει τοπικά ελάχιστα. Αν, ωστόσο, για τα κέντρα και τις ακτίνες γίνει αρχικοποίηση με καλές τιμές τότε η gradient descent θα οδηγήσει σε καλές λύσεις. Για το λόγο αυτό, καθώς **επιθυμούμε τα κέντρα των κρυφών νευρώνων RBF να τοποθετούνται σε κέντρα ομάδων** που υπάρχουν στα δεδομένα εκπαίδευσης, η μέθοδος **εκπαίδευσης δύο σταδίων** προτιμάται.

Στη μέθοδο **εκπαίδευσης δύο σταδίων**:

**ΣΤΑΔΙΟ 1**:
- Αρχικά, χρησιμοποιείται το σύνολο των δεδομένων εκπαίδευσης για τον καθορισμό των νευρώνων RBF $X=\{ \vec{x}^{(1)}, \vec{x}^{(2)}, ..., \vec{x}^{(n)} \}$, όπου έχουμε $n$ τον αριθμό δειγματικά στοιχεία συνόλου εκπαίδευσης και το $i$-οστό διάνυσμα εισόδου είναι το $\vec{x}^{(i)} = (x^{(i)}_1,x^{(i)}_2,...,x^{(i)}_d)^T$
- Αγνοειται η πληροφορία σχετικά με την κατηγορία κάθε προτύπου εκπαίδευσης
- Εφαρμόζεται κάποια μέθοδος ομαδοποίησης/συσταδοποίησης, λόγου χάρη k-means. Από εκεί, προκύπτουν τα κέντρα των ομάδων $\vec{w_j}$, ενώ ο καθορισμός της ακτίνας γίνεται υπολογίζοντας τη διασπορά των δεδομένων κάθε ομάδας $\sigma_{\mathrm{j}}^{2}=\frac{1}{\mathrm{~N}_{\mathrm{j}}} \sum_{\mathrm{χ}^{\mathrm{n}} \in \mathrm{O}_{\mathrm{j}}}\left\|\mathrm{x}^{\mathrm{n}}-\mathrm{w}_{\mathrm{j}}\right\|^{2}$

**ΣΤΑΔΙΟ 2**
Για το δεύτερο στάδιο μπορούν να γίνουν δύο επιλογές:
1. Τα βάρη $w_{ij}$ και $\sigma_j$ παραμένουν σταθερά στις τιμές που καθορίστηκαν στο 1ο στάδιο και απομένει η εύρεση μόνο των βαρών $v_{ij}$ και των biases $v_{i0}$ του επιπέδου εξόδου. Στην περίπτωση αυτή προσδιορίζονται τα βάρη αυτά με τη μέθοδο gradient descent.
2. Εφαρμογή gradient descent όπως στην περίπτωση της ενιαίας εκπαίδευσης, αρχικοποιώντας όμως τα κέντρα και τις ακτίνες RBF στις τιμές που προέκυψαν από το 1ο στάδιο.


**Ερώτηση 3**:
Επιχειρήστε να αλλάξετε την τιμή του k σε 10 και στη συνέχεια σε 50. Τι παρατηρείτε ως προς την ακρίβεια των αποτελεσμάτων;

k=10

In [25]:
K_cent= 10
km= KMeans(n_clusters= K_cent, max_iter= 100)
km.fit(X_train)
cent= km.cluster_centers_

max=0
for i in range(K_cent):
	for j in range(K_cent):
		d= np.linalg.norm(cent[i]-cent[j])
		if(d> max):
			max= d
d= max

sigma= d/math.sqrt(2*K_cent)

shape= X_train.shape
row= shape[0]
column= K_cent
F= np.empty((row,column), dtype= float)

for i in range(row):
  for j in range(column):
    dist= np.linalg.norm(X_train[i]-cent[j])
    F[i][j]= math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))

FTG= np.dot(F.T,F)
FTG_inv= np.linalg.inv(FTG)
fac= np.dot(FTG_inv,F.T)
W= np.dot(fac,y_train)

row= X_test.shape[0]
column= K_cent
F_test= np.empty((row,column), dtype= float)
for i in range(row):
	for j in range(column):
		dist= np.linalg.norm(X_test[i]-cent[j])
		F_test[i][j]= math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))


prediction= np.dot(F_test,W)
prediction= 0.5*(np.sign(prediction-0.5)+1)

score= accuracy_score(prediction,y_test)
print(score.mean())

0.8914209115281502


Παρατηρούμε πως αυξάνοντας τον αριθμό των κέντρων στον k-means, k από 8 σε 10, το score αυξάνεται από 88.77% σε 89.14%.

k=50

In [26]:
K_cent= 50
km= KMeans(n_clusters= K_cent, max_iter= 100)
km.fit(X_train)
cent= km.cluster_centers_

max=0
for i in range(K_cent):
	for j in range(K_cent):
		d= np.linalg.norm(cent[i]-cent[j])
		if(d> max):
			max= d
d= max

sigma= d/math.sqrt(2*K_cent)

shape= X_train.shape
row= shape[0]
column= K_cent
F= np.empty((row,column), dtype= float)

for i in range(row):
  for j in range(column):
    dist= np.linalg.norm(X_train[i]-cent[j])
    F[i][j]= math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))

FTG= np.dot(F.T,F)
FTG_inv= np.linalg.inv(FTG)
fac= np.dot(FTG_inv,F.T)
W= np.dot(fac,y_train)

row= X_test.shape[0]
column= K_cent
F_test= np.empty((row,column), dtype= float)
for i in range(row):
	for j in range(column):
		dist= np.linalg.norm(X_test[i]-cent[j])
		F_test[i][j]= math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))


prediction= np.dot(F_test,W)
prediction= 0.5*(np.sign(prediction-0.5)+1)

score= accuracy_score(prediction,y_test)
print(score.mean())

0.8949061662198391


Αρχικά, παρατηρήσαμε ότι αυξάνοντας το k περαιτέρω από 10 σε 50, ο χρόνος εκτέλεσης αυξήθηκε σημαντικά, καθώς αυξάνουμε τον αριθμό των κρυφών νευρώνων και συναρτήσεων βάσης RBF, άρα και των παραμέτρων των δικτύων.

Ωστόσο το score από 89.14% πήγε σε 89.49%, μια πολύ μικρή αύξηση που μας δείχνει πως ενδεχομένως ο βέλτιστος αριθμός k είναι πιο κοντά στο 10. Αυτός ο αριθμός (k) είναι ουσιαστικά ο αριθμός ομάδων στον οποίων εμπίπτουν τα δεδομένα εκπαίδευσης.